# Derive Top N Portfolio

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

In [8]:
import os
import sys
from datetime import datetime, date, timedelta

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
import rezaware as reza
from utils.modules.etl.load import sparkDBwls as sdb
from utils.modules.etl.transform import sparkCleanNRich as scne
from mining.modules.assets.etp import dataPrep as prep
# from utils.modules.ml.timeseries import rollingstats as stats

''' restart initiate classes '''
if debug:
    import importlib
    reza = importlib.reload(reza)
    sdb = importlib.reload(sdb)
    scne = importlib.reload(scne)
    prep = importlib.reload(prep)
#     stats= importlib.reload(stats)
    
__desc__ = "analyze crypto market capitalization time series data"
# clsSDB = sdb.SQLWorkLoads(desc=__desc__)
clsSCNR=scne.Transformer(desc=__desc__)
# clsStat=stats.RollingStats(desc=__desc__)
clsPrep =prep.Warehouse(desc=__desc__)
''' optional - if not specified class will use the default values '''
# prop_kwargs = {"WRITE_TO_TMP":True,   # necessary to emulate the etl dag
#               }
print("\nClass initialization and load complete!")

All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
All functional SPARKDBWLS-libraries in LOAD-package of ETL-module imported successfully!
All functional SPARKCLEANNRICH-libraries in TRANSFORM-package of ETL-module imported successfully!
All functional DATAPREP-libraries in ETP-package of ASSETS-module imported successfully!
sparkNoSQLwls Class initialization complete
dataPrep Class initialization complete

Class initialization and load complete!


## Read data from mcap_past
We apply a query to select assets with mcap > 1.0 million. Any missing values are imputed with the mean value.

In [9]:
_from_date = '2022-01-05'
_to_date = '2022-01-10'
# _query = "select * from warehouse.mcap_past "+\
#         f"where mcap_date >= '{_from_date}' and "+\
#         f"mcap_date <= '{_to_date}'"
_query = "select * from warehouse.mcap_past wmp "+\
        f"where wmp.mcap_date between '{_from_date}' and '{_to_date}' "+\
        f"and wmp.mcap_value > 1000000 "+\
        f"and wmp.mcap_value = (select MAX(mcap_value ) "+\
        f"from warehouse.mcap_past where wmp.asset_name=asset_name)"
_kwargs = {
    "TABLENAME":'warehouse.mcap_past',
    "COLUMN":'mcap_date',
    "FROMDATETIME":_from_date,
    "TODATETIME":_to_date,
    "PARTITIONS":2,
    "AGGREGATE":'avg',
    "PIVCOLUMNS":['dxd','sofi','wsn','xmx','uqc','btr','unic','nex','noia',
                  'hanu','aca','bbs','xvs','pnd','shake','stpl','dtx','tethys',
                  'kyoko','boba','nlife','rare','eved','yfl','fkx','flixx',
                  'drk','meto','glide','shr','tetu','mft','cmerge','shmn','tronpad']
}

# print(clsSpark.dbSchema)
mcap_sdf = clsPrep.read_n_clean_mcap(query=_query,**_kwargs)
# mcap_sdf = clsROR.read_n_clean_mcap(**_kwargs)

print("Loaded %d rows and %d columns" % (mcap_sdf.count(),len(mcap_sdf.columns)))

Wait a moment, retrieving data ...


23/02/08 16:59:52 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB


23/02/08 17:00:03 WARN DAGScheduler: Broadcasting large task binary with size 6.2 MiB


23/02/08 17:01:57 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


23/02/08 17:02:56 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


23/02/08 17:03:57 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


Loaded 235553 rows and 15 columns


In [13]:
from pyspark.sql import functions as F
mcap_sdf.select(F.col('uuid'),
                F.col('mcap_date'),
                F.col('asset_name'),
                F.col('mcap_value'),
                F.col('mcap_rank'),
                F.col('mcap_dt')).show(n=3)

23/02/08 18:24:09 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB


+--------------------+----------+----------------+--------------------+---------+-------------------+
|                uuid| mcap_date|      asset_name|          mcap_value|mcap_rank|            mcap_dt|
+--------------------+----------+----------------+--------------------+---------+-------------------+
|6397ab7b7cc473c58...|2022-01-05|synthetify_token|14232382.38223430...|     null|2022-01-05 00:00:00|
|639720647cc473c58...|2022-01-05|            celo|2214150436.063860...|     null|2022-01-05 00:00:00|
|639720647cc473c58...|2022-01-05|            celo|2193746148.681650...|     null|2022-01-05 00:00:00|
+--------------------+----------+----------------+--------------------+---------+-------------------+
only showing top 3 rows



## Compute LogROR for all assets

In [ ]:
kwargs={
    "PREVALCOLNAME":'mcap_prev_val',
    "DIFFCOLNAME":'mcap_diff',
    "LOGCOLNAME":'log_ror'
}
_mcap_log_ror, _log_col = clsPrep.get_log_ror(
    data=mcap_sdf,
    num_col_name="mcap_value",
    part_column ='asset_name',
    **kwargs,
)

In [18]:
_mcap_log_ror.select(F.col('uuid'),
                     F.col('asset_name'),
                     F.col('mcap_date'),
                     F.col('mcap_value'), 
                     F.col('log_ror'))\
            .filter((F.col('log_ror').isNotNull()) & 
                    (F.col('asset_name')=='axial_token'))\
            .show(n=3)

23/02/08 18:35:37 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


23/02/08 18:35:44 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


23/02/08 18:35:52 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB


23/02/08 18:35:53 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB
+--------------------+-----------+----------+--------------------+--------------------+
|                uuid| asset_name| mcap_date|          mcap_value|             log_ror|
+--------------------+-----------+----------+--------------------+--------------------+
|                null|axial_token|2022-01-05|4702556.663127339...| -0.6692981455062752|
|6396fb727cc473c58...|axial_token|2022-01-05|5188795.589442330...|-0.04273233278400148|
|6396fb727cc473c58...|axial_token|2022-01-05|5740784.796051540...|-0.03844934887355...|
+--------------------+-----------+----------+--------------------+--------------------+
only showing top 3 rows



23/02/08 17:21:21 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


23/02/08 17:22:43 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


23/02/08 17:23:47 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


23/02/08 17:24:00 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


23/02/08 17:24:14 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB


23/02/08 17:24:19 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB


+--------------------+-----------+-----------+------------+----------+--------------------+---------+--------------------+----------+------------+-----------+-----------+-------------+-------------+-------------------+--------------------+--------------------+-------------------+
|                uuid|data_source| asset_name|asset_symbol| mcap_date|          mcap_value|mcap_rank|          created_dt|created_by|created_proc|modified_dt|modified_by|modified_proc|deactivate_dt|            mcap_dt| mcap_value_prev_val|           mcap_diff|            log_ror|
+--------------------+-----------+-----------+------------+----------+--------------------+---------+--------------------+----------+------------+-----------+-----------+-------------+-------------+-------------------+--------------------+--------------------+-------------------+
|                null|       null|axial_token|        null|2022-01-05|4702556.663127339...|     null|                null|      null|        null|       null

## Weighted Portfolio

In [11]:
_cols={
    "NAMECOLUMN":'asset_name',
    "DATECOLUMN":'mcap_date',
    "RORCOLUMN" :_log_col,
    "MCAPCOLUMN":'mcap_value',
    "WEIGHTCOLUMN":'weights',
}
_l_exp_wts,_cols_dict=clsMPT.get_weighted_mpt(
    data=_mcap_log_ror,
    cols_dict=_cols,
#     date_col='mcap_date',
#     val_col='log_ror',
#     name_col='asset_name',
    topN=3,
    size=5,
    **_kwargs,
)
# print("Dates: %s" % str(_wr_dates))
# print("Data : row=%d columns=%d" % (_wr_data.count(),len(_wr_data.columns)))
_l_exp_wts

23/01/28 09:49:03 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


23/01/28 09:49:08 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


23/01/28 09:49:13 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB


23/01/28 09:51:38 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


23/01/28 09:51:43 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


...

23/01/28 10:15:31 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB


23/01/28 10:15:33 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB


23/01/28 10:15:34 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB


23/01/28 10:15:36 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB


[{'mcap_date': datetime.date(2022, 1, 6),
  'asset_name': ['charg_coin', 'roge', 'neurotoken'],
  'log_ror': [0.25040658289005097, 0.2300847277541936, 0.22315158781277547],
  'weights': [0.5934458610550608, 0.11493828956433183, 0.2916158493806074],
  'mcap_value': [Decimal('5220183.74772765300000000000'),
   Decimal('2145261.20389485800000000000'),
   Decimal('1332808.07022394360000000000')]},
 {'mcap_date': datetime.date(2022, 1, 7),
  'asset_name': ['robotina', 'sentinel_group', 'gravity_finance'],
  'log_ror': [0.9659091703444093, 0.415784053356342, 0.24974754544640954],
  'weights': [0.6295957738354031, 0.05237806329355537, 0.3180261628710416],
  'mcap_value': [Decimal('1018345.62651913000000000000'),
   Decimal('5870017.17622237100000000000'),
   Decimal('2031961.70641057660000000000')]},
 {'mcap_date': datetime.date(2022, 1, 8),
  'asset_name': ['primecoin', 'neurochain', 'bitcoin_scrypt'],
  'log_ror': [0.45615688364912843, 0.4011424725185939, 0.28212772816557075],
  'weights': 

## Write MPT to MongoDB

* Collection name = "mpt."+date(YYYY-MM-DD)
* document structure: \_id, date, asset, mcap.value, mcap.weight, mcap.ror

In [20]:
_kwargs = {
    "DESTINDBNAME":'tip-daily-mpt',
    "COLLPREFIX" : 'mpt.for'
}
mpt_list_ = clsMPT.write_mpt_to_db(
    mpt_data=_l_exp_wts,
    cols_dict=_cols_dict,
    **kwargs,
)
print("Upsert %d documents" % len(mpt_list_))

Upsert 5 documents
